### <font color="brown">Relational Databases Continued</font>

---

#### Nobel Prize Winners Database Version 2

---

In case you lost track of the database state at the end of the previous (Apr 11) class, you can get to it by loading
the <tt>nobelsv2.sql</tt> file:

<pre>
venugopa@data8:~/cs210_s24/lectures$ mysql venugopa_nobels < nobelsv2.sql
</pre>

Note: the nobelsv2.sql file must be in the folder where you are executing the command.

---

In [16]:
# import connector modules
from mysql.connector import connect, Error

In [44]:
# connect to nobels database
try:
    mydb = connect(unix_socket='/run/mysqld/mysqld.sock', database="venugopa_nobels")
    cursor = mydb.cursor()
except Error as e:
    print(e)

**Insert multiple rows in laureate table**

In [18]:
# set up sql template to add laureate
add_laureate = "insert into laureate values (%s, %s, %s, %s, %s)"

In [19]:
# let's add all 2021 Economics laureates in one shot
# year_cat_id=2 is same for all of them, but share and motiv_id vary
laureates = [('David','Card',2,2,2), ('Joshua','Angrist',4,2,3), ('Guido','Imbens',4,2,3)]
cursor.executemany(add_laureate, laureates)
print(cursor.rowcount,'record(s) inserted')
mydb.commit()  # update the database

3 record(s) inserted


In [20]:
# check the laureate table
query = 'select * from laureate' 
cursor.execute(query)
res = cursor.fetchall()
for row in res:
    print(row)

('Benjamin', 'List', 2, 1, 1)
('David', 'MacMillan', 2, 1, 1)
('David', 'Card', 2, 2, 2)
('Joshua', 'Angrist', 4, 2, 3)
('Guido', 'Imbens', 4, 2, 3)


---

##### <font color="brown">We're going to switch over to terminal to delete, from yearcat, all rows except the first two (for Chemistry and Economics). </font>
##### <font color="brown"> Before we do that, we should close the cursor and the connection.</font>

In [21]:
cursor.close()

True

In [22]:
mydb.close()

<b>In Terminal:
<pre>
venugopa@data8:~/cs210_s24/lectures$ mysql venugopa_nobels
...

MariaDB [venugopa_nobels]> select * from yearcat;
+----+------+------------+
| id | year | category   |
+----+------+------------+
|  1 | 2021 | Chemistry  |
|  2 | 2021 | Economics  |
|  3 | 2021 | Literature |
|  4 | 2021 | Physics    |
|  5 | 2021 | Peace      |
|  6 | 2021 | Medicine   |
+----+------+------------+
6 rows in set (0.00 sec)

MariaDB [venugopa_nobels]> delete from yearcat where id > 2;
Query OK, 3 rows affected (0.00 sec)

MariaDB [venugopa_nobels]> select * from yearcat;
+----+------+-----------+
| id | year | category  |
+----+------+-----------+
|  1 | 2021 | Chemistry |
|  2 | 2021 | Economics |
+----+------+-----------+
2 rows in set (0.00 sec)

MariaDB [venugopa_nobels]> select * from winners where year=2021 and category='Physics';
+------+----------+---------+------------+---------------------------------------------------------------------------------------------------------------------+-------+
| year | category | fname   | lname      | motivation                                                                                                          | share |
+------+----------+---------+------------+---------------------------------------------------------------------------------------------------------------------+-------+
| 2021 | Physics  | Syukuro | Manabe     | for the physical modelling of Earth’s climate, quantifying variability and reliably predicting global warming       |     4 |
| 2021 | Physics  | Klaus   | Hasselmann | for the physical modelling of Earth’s climate, quantifying variability and reliably predicting global warming       |     4 |
| 2021 | Physics  | Giorgio | Parisi     | for the discovery of the interplay of disorder and fluctuations in physical systems from atomic to planetary scales |     2 |
+------+----------+---------+------------+---------------------------------------------------------------------------------------------------------------------+-------+

MariaDB [venugopa_nobels]> exit
Bye

venugopa@data8:~/cs210_s24/lectures$
</pre>

---

#### Back to Python programming

##### <font color="brown">Make sure to redo the mydb connection and set up cursor, like we did initially. Just jump to and re-execute that cell, then jump back here.</font>

---

#### <font color="brown">Adding to all three tables (yearcat, contribution, and laureate), retaining id from yearcat and contribution for add to laureate</font>

##### <font color="brown">Add Physics laureates for 2021</font>

**Set up sql templates and values**

In [24]:
# set up sql template and values for yearcat table
add_yearcat = 'insert into yearcat (year,category) values (%s,%s)'

# set up sql template and values for contribution table
add_contribution = "insert into contribution (motivation) values (%s)"

str1 = 'for the physical modelling of Earth’s climate, ' 
str2 = 'quantifying variability and reliably predicting global warming'
share4_contrib = str1 + str2        

str1 = 'for the discovery of the interplay of disorder and fluctuations '
str2 = 'in physical systems from atomic to planetary scales'
share2_contrib = str1 + str2

##### **Add to yearcat and get last added id using cursor.lastrowid. Similarly for contribution. Then add laureates.**
https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-lastrowid.html

In [25]:
# add yearcat, and get last added id
cursor.execute(add_yearcat, (2021,'Physics'))
mydb.commit()  # update the database
yc_id = cursor.lastrowid
print(yc_id)

7


**The id is 7 because the max id of all rows that had been previously added was 6 (Medicine). Subsequently, we deleted the ids 3, 4,5,6 but that does not change the fact that max id ever added was 6. So a new add gets id=6+1=7.**

In [26]:
# add share4 contributon and get last added id
cursor.execute(add_contribution, (share4_contrib,))  # Note the tuple syntax!! Only one value to be added
mydb.commit()  # update the database
contrib_id = cursor.lastrowid
print(contrib_id)

4


In [27]:
# now add the two laureates at a share of 4 apiece
laureates = [('Syukuro','Manabe',4,yc_id,contrib_id), ('Klaus','Hasselmann',4,yc_id,contrib_id)]
cursor.executemany(add_laureate,laureates)
mydb.commit()

In [28]:
# add share2 contributon and get last added id
cursor.execute(add_contribution, (share2_contrib,))  
mydb.commit()  # update the database
contrib_id = cursor.lastrowid

In [29]:
# add the third laureate, with a share of 2
cursor.execute(add_laureate,('Giorgio','Parisi',2,yc_id,contrib_id))
print(cursor.rowcount,'record(s) inserted')
mydb.commit()

1 record(s) inserted


In [30]:
# get contents of laureate table
query = 'select * from laureate'
cursor.execute(query)
res = cursor.fetchall()
for row in res:
    print(row)

('Benjamin', 'List', 2, 1, 1)
('David', 'MacMillan', 2, 1, 1)
('David', 'Card', 2, 2, 2)
('Joshua', 'Angrist', 4, 2, 3)
('Guido', 'Imbens', 4, 2, 3)
('Syukuro', 'Manabe', 4, 7, 4)
('Klaus', 'Hasselmann', 4, 7, 4)
('Giorgio', 'Parisi', 2, 7, 5)


In [31]:
cursor.close()

True

In [32]:
mydb.close()

---

#### <font color="brown">Populating the database with all the data from the nobel prize winners dataset</font>

##### We start from scratch and populate the <tt>yearcat</tt>, <tt>contribution</tt>, and <tt>laureate</tt> tables with the entire nobels JSON dataset

##### <font color="brown">First, open a mysql client session in Terminal, and delete all rows from these tables</font>

<pre>
venugopa@data8:~/cs210_s24/lectures$ mysql venugopa_nobels
...
MariaDB [venugopa_nobels]> delete from yearcat;
ERROR 1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails (`venugopa_nobels`.`laureate`, CONSTRAINT `laureate_ibfk_1` FOREIGN KEY (`year_cat_id`) REFERENCES `yearcat` (`id`))
</pre>
Rows from the <tt>yearcat</tt> table cannot be deleted since they have a key that is referenced in another table.<br>
(Likewise for the <tt>contribution</tt> table.)

So we will need to delete from the <tt>laureate</tt> table first, and then delete from the other two tables.
<pre>
MariaDB [venugopa_nobels]> delete from laureate;
Query OK, 8 rows affected (0.004 sec)

MariaDB [venugopa_nobels]> delete from yearcat;
Query OK, 3 rows affected (0.004 sec)

MariaDB [venugopa_nobels]> delete from contribution;
Query OK, 5 rows affected (0.003 sec)

MariaDB [venugopa_nobels]> select * from yearcat;
Empty set (0.000 sec)

MariaDB [venugopa_nobels]> select * from contribution;
Empty set (0.000 sec)

MariaDB [venugopa_nobels]> select * from laureate;
Empty set (0.000 sec)
</pre>

##### <font color="brown">Then, switch to Python, read the JSON dataset, and insert into the tables</font>

In [36]:
import json, requests

nobel_url = 'http://api.nobelprize.org/v1/prize.json'
resp = requests.get(nobel_url)
nobels = json.loads(resp.text)

In [39]:
for i,prize in enumerate(nobels['prizes']):
    if i > 2:
        break
    print(prize)

{'year': '2023', 'category': 'chemistry', 'laureates': [{'id': '1029', 'firstname': 'Moungi', 'surname': 'Bawendi', 'motivation': '"for the discovery and synthesis of quantum dots"', 'share': '3'}, {'id': '1030', 'firstname': 'Louis', 'surname': 'Brus', 'motivation': '"for the discovery and synthesis of quantum dots"', 'share': '3'}, {'id': '1031', 'firstname': 'Aleksey', 'surname': 'Yekimov', 'motivation': '"for the discovery and synthesis of quantum dots"', 'share': '3'}]}
{'year': '2023', 'category': 'economics', 'laureates': [{'id': '1034', 'firstname': 'Claudia', 'surname': 'Goldin', 'motivation': '"for having advanced our understanding of women’s labour market outcomes"', 'share': '1'}]}
{'year': '2023', 'category': 'literature', 'laureates': [{'id': '1032', 'firstname': 'Jon', 'surname': 'Fosse', 'motivation': '"for his innovative plays and prose which give voice to the unsayable"', 'share': '1'}]}


##### Cycle through the laureates list: 
- When we get to the first instance of a year/category, we insert it into the yearcat table. 
- At the same time, we also insert the corresponding motivation in the contribution table. 
- Using the lastrowid for both, we insert the associated laureate. 
- Then, as long as both year/category and motivation are the same, we keep adding laureates with the same yearcat id and the same motivation id. 
- If the motivation changes, but year/category is the same, we insert a new contribution, get lastrowid for that table, and then keep adding laureates

**Make sure to connect to the database and get a cursor before proceeding!**

In [37]:
# query templates for adding to the tables
add_yearcat = "insert into yearcat (year,category) values (%s,%s)"
add_contribution = "insert into contribution (motivation) values (%s)"
add_laureate = "insert into laureate values (%s, %s, %s, %s, %s)"

In [46]:
prev_year = 0
prev_cat = ''
prev_motiv = ''
for prize in nobels['prizes']:
    if not 'laureates' in prize:
        continue
    year = int(prize['year'])
    cat = prize['category'].capitalize()
    yc_val = (year, cat)
    if year != prev_year or cat != prev_cat:  # switch
        cursor.execute(add_yearcat, yc_val)
        yc_id = cursor.lastrowid
        prev_year = year
        prev_cat = cat
    for winner in prize['laureates']:
        motiv = winner.get('motivation').strip('"')
        if motiv != prev_motiv:
            cursor.execute(add_contribution, (motiv,))
            contrib_id = cursor.lastrowid
            prev_motiv = motiv
        fname = winner.get('firstname')
        share = winner.get('share')
        try:
            lname = winner.get('surname')
        except KeyError:
            lname=None
        
        cursor.execute(add_laureate,(fname,lname,share,yc_id,contrib_id))
        mydb.commit()     

---

**When we created the <tt>laureate</tt> table, we wrote in 'not null' for the <tt>lname</tt> column. That was a mistake since some prizes don't have a lastname. So we need to change the schema of the <tt>laureate</tt> table with an updated version of <tt>lname</tt> to allow null values**

We will do the alteration in Terminal, but we need to first close the connection to the db from the Python program, otherwise there will be a conflict.

In [43]:
cursor.close()
mydb.close()

<pre>
MariaDB [venugopa_nobels]> alter table laureate modify column lname varchar(40);
Query OK, 0 rows affected (0.028 sec)
Records: 0  Duplicates: 0  Warnings: 0

MariaDB [venugopa_nobels]> desc laureate;
+-------------+-------------+------+-----+---------+-------+
| Field       | Type        | Null | Key | Default | Extra |
+-------------+-------------+------+-----+---------+-------+
| fname       | varchar(80) | NO   |     | NULL    |       |
| lname       | varchar(40) | YES  |     | NULL    |       |
| share       | tinyint(4)  | NO   |     | NULL    |       |
| year_cat_id | smallint(6) | NO   | MUL | NULL    |       |
| motiv_id    | smallint(6) | NO   | MUL | NULL    |       |
+-------------+-------------+------+-----+---------+-------+
5 rows in set (0.002 sec)

MariaDB [venugopa_nobels]> 
</pre>

**When we ran cell 46, a bunch of rows were inserted (committed) into <tt>laureate</tt> we hit a winner that had a missing lastname. You can tell how many rows were inserted if you examind the lname column in the <tt>winners</tt> table** 

<pre>
MariaDB [venugopa_nobels]> select lname from winners limit 22;
+--------------+
| lname        |
+--------------+
| Bawendi      |
| Brus         |
| Yekimov      |
| Goldin       |
| Fosse        |
| Mohammadi    |
| Agostini     |
| Krausz       |
| L’Huillier   |
| Karikó       |
| Weissman     |
| Bertozzi     |
| Meldal       |
| Sharpless    |
| Bernanke     |
| Diamond      |
| Dybvig       |
| Ernaux       |
| Bialiatski   |
|              |
|              |
| Aspect       |
+--------------+
</pre>
The 20th row has a null value (shown as a blank), so 19 rows were inserted into <tt>laureates</tt>

**So let's clear out all the tables (except <tt>winners</tt>) before we redo the inserts**

<pre>
MariaDB [venugopa_nobels]> delete from laureate;
Query OK, 1019 rows affected (0.011 sec)

MariaDB [venugopa_nobels]> delete from yearcat;
Query OK, 631 rows affected (0.006 sec)

MariaDB [venugopa_nobels]> delete from contribution;
Query OK, 700 rows affected (0.006 sec)
</pre>

**Reconnect to the db and get cursor, then run cell 46, and return here**

<pre>
MariaDB [venugopa_nobels]> select count(*) from laureate;
+----------+
| count(*) |
+----------+
|     1000 |
+----------+
1 row in set (0.002 sec)
</pre>

---

#### <font color="brown">Transaction, Atomicity, and Rollback<font>

**What if there is a problem in adding to contribution table after adding to yearcat, or in adding to laureate after adding to contribution (and possibly to yearcat)?**

- We need to take into account these possible issues and not add the data if there is a problem. 

- In order to do this, we will need to do what's called a **rollback** which basically cancels any other changes you may have made in the same breath (adding to any of the other tables). 

- The reason is if there is an update to more than one table for any laureate entry, either all updates happen, or none happen - this group of adds is called a **transaction** which is *atomic* (all or nothing). A transaction could also be multiple updates to a single table.

- There is a property called autocommit that can be set to True (if you want to automatically commit after a statement instead of having to explicity call commit()) or False (otherwise). The Python MySQL connector sets it to False by default
https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlconnection-autocommit.html

- This is useful for transactions, because you only want to commit after *all* component updates have been executed without error: the autocommit property is false, and an explicit commit is issued at the end of the transaction.  

**Following is the code rewritten with error checking and rollback.**

- First, clear all tables (except <tt>winners</tt>)
<pre>
MariaDB [venugopa_nobels]> delete from laureate;
Query OK, 1000 rows affected (0.013 sec)

MariaDB [venugopa_nobels]> delete from yearcat;
Query OK, 621 rows affected (0.007 sec)

MariaDB [venugopa_nobels]> delete from contribution;
Query OK, 690 rows affected (0.008 sec)
</pre>
- Then execute the following cell



In [47]:
# code includes rollback
import sys

prev_year = 0
prev_cat = ''
prev_motiv = ''
for prize in nobels['prizes']:
    if not 'laureates' in prize:
        continue
    year = int(prize['year'])
    cat = prize['category'].capitalize()
    yc_val = (year, cat)
    if year != prev_year or cat != prev_cat:  # switch
        try:
            cursor.execute(add_yearcat, yc_val)
            yc_id = cursor.lastrowid
            prev_year = year
            prev_cat = cat
        except:  # don't need to rollback here since nothing has been added
            print(sys.exc_info()[0])
            print(f'Could not add {year}/{cat}')
            continue
    
    for winner in prize['laureates']:
        # motivation
        try:
            motiv = winner.get('motivation').strip('"')
            if motiv != prev_motiv:
                cursor.execute(add_contribution, (motiv,))
                contrib_id = cursor.lastrowid
                prev_motiv = motiv
        except:
            # contribution failed, rollback in case year/cat was added
            mydb.rollback()   # no effect or undo year/cat if added since last commit
            print(f'Could not add motivation: "{motiv} in {year}/{cat}"')
            break
            
        # laureate
        try:
            fname = winner.get('firstname')
            share = winner.get('share')
            try:
                lname = winner.get('surname')
            except KeyError:
                lname=None
            cursor.execute(add_laureate,(fname,lname,share,yc_id,contrib_id))
            mydb.commit()    
        except:
            # laureate failed
            mydb.rollback()   # no effect, or undo contribution, or undo contrib and year/cat
            print(f'Could not add laureate {fname} {lname} in {year}/{cat}')
            break  
          

In [48]:
# check the count in laureate
cursor.execute('select count(*) from laureate')
res = cursor.fetchall()
for row in res:
    print(row)

(1000,)


#### You can do transaction with commit and rollback in the MySQL client terminal:
https://dev.mysql.com/doc/refman/8.0/en/innodb-autocommit-commit-rollback.html

Unlike the Python connector, a connection through the client interface will start with autocommit set to 1 (True) by default. So if you want to process a transaction, you will need to set autocommit to 0 (false) first.

See also: https://dev.mysql.com/doc/refman/8.0/en/commit.html

---

#### <font color="brown">Queries on the nobels database</font>

##### **The table schemas**

<pre>
mysql> desc yearcat;
+----------+----------+------+-----+---------+----------------+
| Field    | Type     | Null | Key | Default | Extra          |
+----------+----------+------+-----+---------+----------------+
| id       | smallint | NO   | PRI | NULL    | auto_increment |
| year     | year     | NO   |     | NULL    |                |
| category | char(10) | NO   |     | NULL    |                |
+----------+----------+------+-----+---------+----------------+
3 rows in set (0.00 sec)

mysql> desc contribution;
+------------+--------------+------+-----+---------+----------------+
| Field      | Type         | Null | Key | Default | Extra          |
+------------+--------------+------+-----+---------+----------------+
| id         | smallint     | NO   | PRI | NULL    | auto_increment |
| motivation | varchar(500) | NO   |     | NULL    |                |
+------------+--------------+------+-----+---------+----------------+
2 rows in set (0.00 sec)

mysql> desc laureate;
+-------------+-------------+------+-----+---------+-------+
| Field       | Type        | Null | Key | Default | Extra |
+-------------+-------------+------+-----+---------+-------+
| fname       | varchar(80) | NO   |     | NULL    |       |
| lname       | varchar(40) | YES  |     | NULL    |       |
| share       | tinyint     | NO   |     | NULL    |       |
| year_cat_id | smallint    | NO   | MUL | NULL    |       |
| motiv_id    | smallint    | NO   | MUL | NULL    |       |
+-------------+-------------+------+-----+---------+-------+
5 rows in set (0.00 sec)
</pre>

---

##### <font color="brown">1. Who were the laureates in 2010, and for what category? (Inner Join)</font>

<pre>
MariaDB [venugopa_nobels]> select fname,lname,category from laureate,yearcat where yearcat.year=2010 and laureate.year_cat_id=yearcat.id;
       
+----------------+--------------+------------+
| fname          | lname        | category   |
+----------------+--------------+------------+
| Richard F.     | Heck         | Chemistry  |
| Ei-ichi        | Negishi      | Chemistry  |
| Akira          | Suzuki       | Chemistry  |
| Peter A.       | Diamond      | Economics  |
| Dale T.        | Mortensen    | Economics  |
| Christopher A. | Pissarides   | Economics  |
| Mario          | Vargas Llosa | Literature |
| Xiaobo         | Liu          | Peace      |
| Andre          | Geim         | Physics    |
| Konstantin     | Novoselov    | Physics    |
| Robert G.      | Edwards      | Medicine   |
+----------------+--------------+------------+       
</pre>

- We are executing what's called an **inner join** on the tables laureate and yearcat, because we are matching the year_cat_id value in the laureate table with the id value in the yearcat table - only the respective rows from these two tables for which these ids match will be selected for the result.

- Since year_cat_id, year, and id are unique column names across the pair of tables, you can omit the qualifers and write a simplified version like this:

<pre>
MariaDB [venugopa_nobels]> select fname,lname,category from laureate,yearcat where yearcat.year=2010 and year_cat_id=id;
</pre>

- Another alternative is to explicity spell out the inner join using the JOIN and ON keywords:

<pre>
MariaDB [venugopa_nobels]> select fname,lname,category
    -> from laureate
    -> join yearcat
    -> on year_cat_id=id
    -> where year=2010;
</pre>

- The order of tables is irrelevant, you can do the following (yearcat join laureate) with the same result:

<pre>
MariaDB [venugopa_nobels]> select fname,lname,category
    -> from yearcat
    -> join laureate
    -> on year_cat_id=id
    -> where year=2010;
</pre>

---

##### <font color="brown">2. Find the year and category for laureates that did not have a last name (null values)</font>

<pre>
MariaDB [venugopa_nobels]> select year,category from yearcat,laureate where year_cat_id=id and lname is null;
+------+----------+
| year | category |
+------+----------+
| 2022 | Peace    |
| 2022 | Peace    |
| 2020 | Peace    |
| 2017 | Peace    |
| 2015 | Peace    |

    ......
    
| 1917 | Peace    |
| 1910 | Peace    |
| 1904 | Peace    |
+------+----------+
 
</pre>

Looks like these are all in the Peace category, but we can verify:
<pre>
MariaDB [venugopa_nobels]> select distinct(category) from yearcat,laureate where year_cat_id=id and lname is null;
+----------+
| category |
+----------+
| Peace    |
+----------+
</pre>


---

##### <font color="brown">3. For what contribution(s) was the Peace prize awarded in 2021? (Join on more than 2 tables)</font>

**Version 1**
<pre>
MariaDB [venugopa_nobels]> select motivation
    -> from contribution,yearcat,laureate
    -> where year=2021 and category='Peace'
    -> and yearcat.id = year_cat_id
    -> and contribution.id = motiv_id;
             
+-----------------------------------------------------------------------------------
| motivation                                                                         
+----------------------------------------------------------------------------------- 
| for their efforts to safeguard freedom of expression, which  is a precondition ... 
| for their efforts to safeguard freedom of expression, which  is a precondition ...
+-----------------------------------------------------------------------------------             
</pre>

- This is an inner join of all three tables
- Since both the contribution and yearcat tables have a column named id, we need to qualify its usage with table names

**Version 2**

The above result is not quite what we want, since it is duplicated. (Two people shared the Peace prize in 2021, for the same contribution.) So we need to change the query to get distinct motivation:
<pre>
MariaDB [venugopa_nobels]> select distinct(motivation)
    -> from contribution,yearcat,laureate
    -> where year=2021 and category='Peace'
    -> and yearcat.id = year_cat_id
    -> and contribution.id = motiv_id;
             
+-----------------------------------------------------------------------------------
| motivation                                                                         
+----------------------------------------------------------------------------------- 
| for their efforts to safeguard freedom of expression, which  is a precondition ... 
+-----------------------------------------------------------------------------------                 
</pre>

**Version 3**

In this version, the contribution and yearcat tables are given alternative short labels for ease of writing out the join conditions:

<pre>
MariaDB [venugopa_nobels]> select distinct(motivation)
    -> from contribution C,yearcat Y,laureate
    -> where year=2021 and category='Peace'
    -> and Y.id = year_cat_id
    -> and C.id = motiv_id;              
</pre>

**Version 4**

Often, *all* tables in a join are labeled with single letters and *all* column names are qualified, regardless of whether or not the names are unique in the set of tables:

<pre>
MariaDB [venugopa_nobels]> select distinct(C.motivation)
    -> from contribution C,yearcat Y,laureate L
    -> where Y.year=2021 and Y.category='Peace'
    -> and Y.id = L.year_cat_id
    -> and C.id = L.motiv_id; 
</pre>
I'll mostly follow this practice going forward.

---

##### <font color="brown">4. How many people shared the Economics prize 2018?</font>

<pre>
MariaDB [venugopa_nobels]> select count(*) from yearcat Y, laureate L where Y.year=2018 and Y.category='Economics' and Y.id=L.year_cat_id;
+----------+
| count(*) |
+----------+
|        2 |
+----------+
</pre>